In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import json
import os
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import re
import seaborn as sns
import matplotlib.pyplot as plt
import swifter
import multiprocessing
import time
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.formula.api as smf
from collections import defaultdict
from dataclasses import dataclass
from torch import nn
import torch.nn.functional as F
import torch
import random
import copy
from tqdm import tqdm

sns.set_style("darkgrid")

# Loading Processed Reviews

In [3]:
# Paths
PROCESSED_FOLDER = './data/processed/'
# PROCESSED_REVIEWS_FILE = 'processed_reviews.csv'
PROCESSED_REVIEWS_FILE = 'processed_reviews_with_sentiment.csv'

In [4]:
reviews = pd.read_csv(os.path.join(PROCESSED_FOLDER, PROCESSED_REVIEWS_FILE))

In [5]:
reviews.head()

,review_id,user_id,item_id,text,rating,tokenized_text,neg_sent_avg,neu_sent_avg,pos_sent_avg,compound_sent_avg
0,255938,0,1,"First things first. My ""reviews"" system is exp...",8,"[['First', 'things', 'first', '.'], ['My', '``...",0.073333,0.757708,0.168937,0.208675
1,259117,1,2,Let me start off by saying that Made in Abyss ...,10,"[['Let', 'me', 'start', 'off', 'by', 'saying',...",0.062741,0.759333,0.177963,0.302804
2,253664,2,3,"Art 9/10: It is great, especially the actions ...",7,"[['Art', '9/10', ':', 'It', 'is', 'great', ','...",0.047278,0.792889,0.159833,0.220767
3,247454,3,4,As someone who loves Studio Ghibli and its mov...,6,"[['As', 'someone', 'who', 'loves', 'Studio', '...",0.055577,0.809192,0.135269,0.187896
4,23791,4,5,code geass is one of those series that everybo...,10,"[['code', 'geass', 'is', 'one', 'of', 'those',...",0.028857,0.723000,0.248143,0.534129


# Converting Data for Modeling

In [6]:
# Convert item_id to 0 indexed
if min(reviews['item_id']) != 0:
    reviews['item_id'] = reviews['item_id'] - 1
    print("Done")

Done


In [7]:
@dataclass
class Review:
    user_id: int
    item_id: int
    rating: int
    text: str
    pos_sent: float
    neg_sent: float
    compound_sent: float

In [8]:
USER_KEY = 'user_id'
ITEM_KEY = 'item_id'
RATING_KEY = 'rating'

In [9]:
reviews.head()

,review_id,user_id,item_id,text,rating,tokenized_text,neg_sent_avg,neu_sent_avg,pos_sent_avg,compound_sent_avg
0,255938,0,0,"First things first. My ""reviews"" system is exp...",8,"[['First', 'things', 'first', '.'], ['My', '``...",0.073333,0.757708,0.168937,0.208675
1,259117,1,1,Let me start off by saying that Made in Abyss ...,10,"[['Let', 'me', 'start', 'off', 'by', 'saying',...",0.062741,0.759333,0.177963,0.302804
2,253664,2,2,"Art 9/10: It is great, especially the actions ...",7,"[['Art', '9/10', ':', 'It', 'is', 'great', ','...",0.047278,0.792889,0.159833,0.220767
3,247454,3,3,As someone who loves Studio Ghibli and its mov...,6,"[['As', 'someone', 'who', 'loves', 'Studio', '...",0.055577,0.809192,0.135269,0.187896
4,23791,4,4,code geass is one of those series that everybo...,10,"[['code', 'geass', 'is', 'one', 'of', 'those',...",0.028857,0.723000,0.248143,0.534129


In [10]:
user_to_reviews = defaultdict(list)
for _, row in reviews.iterrows():
    user_id, item_id, rating, text = row[USER_KEY], row[ITEM_KEY], row[RATING_KEY], row['text']
    pos_sent, neg_sent, compound_sent = row['pos_sent_avg'], row['neg_sent_avg'], row['compound_sent_avg']
    user_to_reviews[user_id].append(Review(user_id, item_id, rating, text, pos_sent, neg_sent, compound_sent))

## Creating the score matrix

In [11]:
# users by items
X = np.zeros(shape=(reviews['user_id'].nunique(), reviews['item_id'].nunique()))

In [12]:
for _, row in reviews.iterrows():
    user_id, item_id, rating = row[USER_KEY], row[ITEM_KEY], row[RATING_KEY]
    X[user_id][item_id] = rating

## Train/Test Split

In [13]:
train_X = copy.deepcopy(X)
valid_X = np.zeros(shape=X.shape)
test_X = np.zeros(shape=X.shape)

for user_id, reviews in user_to_reviews.items():
    # can confirm this actually shuffles properly (this code block works)
    random.shuffle(reviews)

    # Leave one out for valid
    valid_review = reviews[0]
    train_X[valid_review.user_id][valid_review.item_id] = 0
    valid_X[valid_review.user_id][valid_review.item_id] = valid_review.rating
    
    # Leave one out for test
    test_review = reviews[1]
    train_X[test_review.user_id][test_review.item_id] = 0
    test_X[test_review.user_id][test_review.item_id] = test_review.rating
    
    # Rest for train

## Creating bias terms for users / items from the training data

In [14]:
# users
user_to_pos_sent = defaultdict(list)
user_to_neg_sent = defaultdict(list)
user_to_compound_sent = defaultdict(list)

# items
item_to_pos_sent = defaultdict(list)
item_to_neg_sent = defaultdict(list)
item_to_compound_sent = defaultdict(list)

# loadding
for user_id, reviews in user_to_reviews.items():
    for r in reviews:
        # skip if not in train
        if train_X[user_id, r.item_id] == 0:
            continue
        user_to_pos_sent[user_id].append(r.pos_sent)
        user_to_neg_sent[user_id].append(r.neg_sent)
        user_to_compound_sent[user_id].append(r.compound_sent)
        item_to_pos_sent[r.item_id].append(r.pos_sent)
        item_to_neg_sent[r.item_id].append(r.neg_sent)
        item_to_compound_sent[r.item_id].append(r.compound_sent)

In [15]:
# Averaging values to get bias term
def list_mapping_to_float_mapping(hm: dict):
    id_to_sent_term = defaultdict(float)
    for k, v in hm.items():
        id_to_sent_term[k] = np.mean(v)
    return id_to_sent_term

user_to_pos_sent_term = list_mapping_to_float_mapping(user_to_pos_sent)
user_to_neg_sent_term = list_mapping_to_float_mapping(user_to_neg_sent)
user_to_compound_sent_term = list_mapping_to_float_mapping(user_to_compound_sent)
item_to_pos_sent_term = list_mapping_to_float_mapping(item_to_pos_sent)
item_to_neg_sent_term = list_mapping_to_float_mapping(item_to_neg_sent)
item_to_compound_sent_term = list_mapping_to_float_mapping(item_to_compound_sent)

## GMF, MLP, and NeuMF

In [16]:
def l2_regularization(values):
    return torch.sum(torch.square(values))

 
class GMF(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=20, regularization_constant=1e-6, sentiment_regularization_constant=1e-6, eps=1e-8):
        super().__init__()
        self.user_factors = nn.Embedding(num_users, embedding_dim)
        self.item_factors = nn.Embedding(num_items, embedding_dim)
        self.regularization_constant = regularization_constant
        self.sentiment_regularization_constant = sentiment_regularization_constant # unused
        self.eps = eps
        
    def forward(self, users: torch.LongTensor, items: torch.LongTensor):
        # (users, emb_dim) * (items, emb_dim) = (interactions, emb_dim)
        result_tensor = self.user_factors(users) * self.item_factors(items)
        user_latent_factors = self.user_factors(users)
        item_latent_factors = self.item_factors(items)
        pred_ratings = user_latent_factors @ item_latent_factors.T
        pred_ratings = 1 + 9 * torch.sigmoid(pred_ratings)
        return pred_ratings.diagonal()
    
    def loss(self, pred_rating: torch.LongTensor, rating: torch.LongTensor, rmse=False):
        if rmse:
            loss = torch.sqrt(F.mse_loss(pred_rating, rating) + self.eps)
        else:
            loss = F.mse_loss(pred_rating, rating) + self.eps
        
        # L2 Regularization
        sum_of_squared_values = l2_regularization(self.user_factors.weight) + l2_regularization(self.item_factors.weight)
        l2_penalty = (1/len(rating)) * self.regularization_constant * sum_of_squared_values

        # Total Loss
        total_loss = loss + l2_penalty
        return total_loss
    
    def predict_single_interaction(self, user_id: int, item_id: int):
        user = torch.LongTensor([user_id]).cuda()
        item = torch.LongTensor([item_id]).cuda()
        return self.forward(user, item)

class MLP(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=20, regularization_constant=1e-6, sentiment_regularization_constant=1e-6, eps=1e-8):
        super().__init__()
        self.user_factors = nn.Embedding(num_users, embedding_dim)
        self.item_factors = nn.Embedding(num_items, embedding_dim)
        self.regularization_constant = regularization_constant
        self.sentiment_regularization_constant = sentiment_regularization_constant # unused
        self.eps = eps
        
        # MLP layers
        self.fc1 = nn.Linear(2*embedding_dim, 128)
        self.relu1 = nn.LeakyReLU()
        self.fc2 = nn.Linear(128, 10)
        self.relu2 = nn.LeakyReLU()
        self.fc3 = nn.Linear(10, 1)
        
        
    def forward(self, users: torch.LongTensor, items: torch.LongTensor):
        # user is shape (users, 1)
        # item is shape (items, 1)
        # embedding output shape is (*, emb_dim) = (users/items, emb_dim)
        user_latent_factors = self.user_factors(users)
        item_latent_factors = self.item_factors(items)
        
        # Concat latent facts together => (*, 2*emb_dim)
        user_item_latent_factors = torch.cat((user_latent_factors, item_latent_factors), dim=1)

        # FC takes (*, in_dim) and outputs (*, out_dim)
        output = self.fc1(user_item_latent_factors)
        output = self.relu1(output)
        output = self.fc2(output)
        output = self.relu2(output)
        output = self.fc3(output)
        
        # Clip in the desired range
        pred_ratings = 1 + 9 * torch.sigmoid(output).squeeze()  

        return pred_ratings
    
    def loss(self, pred_rating: torch.LongTensor, rating: torch.LongTensor, rmse=False):
        if rmse:
            loss = torch.sqrt(F.mse_loss(pred_rating, rating) + self.eps)
        else:
            loss = F.mse_loss(pred_rating, rating) + self.eps
        
        
        # L2 Regularization
        sum_of_squared_values = l2_regularization(self.user_factors.weight) + l2_regularization(self.item_factors.weight)
        l2_penalty = (1/len(rating)) * self.regularization_constant * sum_of_squared_values
        
        # Total Loss
        total_loss = loss + l2_penalty
        return total_loss
    
    def predict_single_interaction(self, user_id: int, item_id: int):
        user = torch.LongTensor([user_id]).cuda()
        item = torch.LongTensor([item_id]).cuda()
        return self.forward(user, item)

    
class NeuMF(nn.Module):
    def __init__(self, num_users, num_items, MF_embedding_dim=20, MLP_embedding_dim=20, alpha=0.5, regularization_constant=1e-6, sentiment_regularization_constant=1e-6, eps=1e-8):
        super().__init__()
        # User factors
        self.MF_user_factors = nn.Embedding(num_users, MF_embedding_dim)
        self.MLP_user_factors = nn.Embedding(num_users, MLP_embedding_dim)
        
        # Item Factors
        self.MF_item_factors = nn.Embedding(num_users, MF_embedding_dim)
        self.MLP_item_factors = nn.Embedding(num_users, MLP_embedding_dim)

        
        # GMF Layer
        # element wise product
        
        # MLP Layers
        self.fc1 = nn.Linear(2*MLP_embedding_dim, 128)
        self.relu1 = nn.LeakyReLU()
        self.fc2 = nn.Linear(128, 10)
        self.relu2 = nn.LeakyReLU()
        self.fc3 = nn.Linear(10, 1)

        # Trainable NeuMF Layer (in the original paper it's just a hyperparameter)
        # Initialize to [0.5, 0.5]
        self.neumf_layer = nn.Linear(2, 1)
        self.neumf_layer.weight.data = torch.Tensor([[0.5, 0.5]]).to(device)
        
        # alpha * MF + (1-alpha) * mlp
        self.alpha = alpha
        
        ## The usual constants
        self.regularization_constant = regularization_constant
        self.sentiment_regularization_constant = sentiment_regularization_constant # unused
        self.eps = eps
        
    def forward(self, users: torch.LongTensor, items: torch.LongTensor):
        # There are all (users x emb_dim), as usual
        MF_latent_user_factors = self.MF_user_factors(users)
        MLP_latent_user_factors = self.MLP_user_factors(users)
        
        MLP_latent_item_factors = self.MLP_item_factors(items)
        MF_latent_item_factors = self.MF_item_factors(items)
        
        # GMF Layer
#         gmf_output = (MF_latent_user_factors @ MF_latent_item_factors.T).diagonal().unsqueeze(1)
        gmf_output = 1 + 9 * torch.sigmoid((MF_latent_user_factors @ MF_latent_item_factors.T).diagonal())
        
        # MLP Layers
        concat_latent_factors = torch.cat((MLP_latent_user_factors, MLP_latent_item_factors), dim=1)
        mlp_output = self.fc1(concat_latent_factors)
        mlp_output = self.relu1(mlp_output)
        mlp_output = self.fc2(mlp_output)
        mlp_output = self.relu2(mlp_output)
        mlp_output = self.fc3(mlp_output)
        mlp_output = (1 + 9 * torch.sigmoid(mlp_output)).squeeze()
        
#         print(gmf_output, mlp_output)
#         print(gmf_output.shape, mlp_output.shape)
        # NeuMF Layer
#         combined_output = torch.cat((gmf_output, mlp_output), dim=1)
#         pred_ratings = self.neumf_layer(combined_output).squeeze()

        return torch.add(self.alpha * gmf_output, (1 - self.alpha) * mlp_output) 
    

    
    def loss(self, pred_rating: torch.LongTensor, rating: torch.LongTensor, rmse=False):
        if rmse:
            loss = torch.sqrt(F.mse_loss(pred_rating, rating) + self.eps)
        else:
            loss = F.mse_loss(pred_rating, rating) + self.eps
        
        # L2 Regularization
        sum_of_squared_values = l2_regularization(self.MF_user_factors.weight) + l2_regularization(self.MF_item_factors.weight)
        sum_of_squared_values += l2_regularization(self.MLP_user_factors.weight)
        sum_of_squared_values += l2_regularization(self.MLP_item_factors.weight)
        for f in [self.fc1, self.fc2, self.fc3]:
            sum_of_squared_values += l2_regularization(f.weight)
        l2_penalty = (1/len(rating)) * self.regularization_constant * sum_of_squared_values

        # Total Loss
        total_loss = loss + l2_penalty
        return total_loss
    
    def predict_single_interaction(self, user_id: int, item_id: int):
        user = torch.LongTensor([user_id, user_id+1]).cuda()
        item = torch.LongTensor([item_id, item_id+1]).cuda()
        return self.forward(user, item)

In [17]:
def eval_MSE_loss(eval_X, model, round_digits=3):
    """Uses reduction mean"""
    user_ids_list, item_ids_list = eval_X.nonzero()
    gt_ratings = torch.FloatTensor([eval_X[user_id, item_id] for user_id, item_id in zip(user_ids_list, item_ids_list)]).to(device)
    curr_users_tensor = torch.LongTensor(user_ids_list).to(device)
    curr_items_tensor = torch.LongTensor(item_ids_list).to(device)
    pred_ratings = model.forward(curr_users_tensor, curr_items_tensor)
    
    return round(F.mse_loss(pred_ratings, gt_ratings).item(), 3)

def eval_RMSE_loss(eval_X, model):
    """Uses reduction mean"""
    user_ids_list, item_ids_list = eval_X.nonzero()
    gt_ratings = torch.FloatTensor([eval_X[user_id, item_id] for user_id, item_id in zip(user_ids_list, item_ids_list)]).to(device)
    curr_users_tensor = torch.LongTensor(user_ids_list).to(device)
    curr_items_tensor = torch.LongTensor(item_ids_list).to(device)
    pred_ratings = model.forward(curr_users_tensor, curr_items_tensor)
    
    return round(torch.sqrt(F.mse_loss(pred_ratings, gt_ratings)).item(), 3)

def eval_MAE_loss(eval_X, model):
    """Uses reduction mean"""
    user_ids_list, item_ids_list = eval_X.nonzero()
    gt_ratings = torch.FloatTensor([eval_X[user_id, item_id] for user_id, item_id in zip(user_ids_list, item_ids_list)]).to(device)
    curr_users_tensor = torch.LongTensor(user_ids_list).to(device)
    curr_items_tensor = torch.LongTensor(item_ids_list).to(device)
    pred_ratings = model.forward(curr_users_tensor, curr_items_tensor)
    
    return round(F.l1_loss(pred_ratings, gt_ratings).item(), 3)


def train_v2(train_X, valid_X, model, optimizer, n_epochs=10, batch_size=5, rmse=False):
    """Training Function, calculates training and validation loss"""
    
    for epoch in (range(1, n_epochs+1)):
        users, items = train_X.nonzero()
        num_examples = len(users)
        permuted_indices = np.random.permutation(num_examples)
        users, items = users[permuted_indices], items[permuted_indices]
        

        total_train_loss = 0
        
        
        for i in tqdm(range(num_examples // batch_size)):
            user_ids_list = users[i*batch_size:i*batch_size+batch_size]
            item_ids_list = items[i*batch_size:i*batch_size+batch_size]

            # Set gradients to zero
            optimizer.zero_grad()

            # Turn data into tensors
            ratings = torch.FloatTensor([train_X[user_id, item_id] for user_id, item_id in zip(user_ids_list, item_ids_list)]).to(device)
            curr_users_tensor = torch.LongTensor(user_ids_list).to(device)
            curr_items_tensor = torch.LongTensor(item_ids_list).to(device)

            # Predict and calculate loss
            pred_ratings = model.forward(curr_users_tensor, curr_items_tensor)
            assert pred_ratings.shape == ratings.shape
            
            ## SELECTING LOSS HERE
            # loss = model.loss(pred_rating, rating)
            loss = model.loss(pred_ratings, ratings, rmse=rmse)

            # Backpropagate
            loss.backward()

            # Update the parameters
            optimizer.step()

            # MSE Loss w/o regularization (just for status updates)
            total_train_loss += F.mse_loss(pred_ratings, ratings, reduction='sum')

        # Computing validation loss for display
        total_valid_loss = eval_MSE_loss(valid_X, model)
        total_valid_RMSE_loss = eval_RMSE_loss(valid_X, model)
        total_valid_MAE_loss = eval_MAE_loss(valid_X, model)
        
        print(f"Epoch {epoch} MSE Loss: {round(total_train_loss.item() / (batch_size * (num_examples//batch_size)), 3)}, valid MSE Loss: {total_valid_loss}, valid RMSE Loss: {total_valid_RMSE_loss}, valid MAE Loss: {total_valid_MAE_loss}")

In [18]:
def eval_model(test_X, model):
    total_test_loss = eval_MSE_loss(test_X, model)
    total_test_RMSE_loss = eval_RMSE_loss(test_X, model)
    total_test_MAE_loss = eval_MAE_loss(test_X, model)
    print(f"test MSE Loss: {total_test_loss}, test RMSE Loss: {total_test_RMSE_loss}, test MAE Loss: {total_test_MAE_loss}")

## Config Cuda

In [ ]:
!nvidia-smi

In [20]:
## Forcing GPU
assert torch.cuda.is_available()
torch.cuda.set_device("cuda:7")
device = torch.device("cuda:7")
a = torch.tensor([[1., 2.], [3., 4.]]).to(device)
a

tensor([[1., 2.],
        [3., 4.]], device='cuda:7')

## Training GMF and MLP Independently
As per the NCF paper, we must train GMF and MLP separately. Then, we initialize NeuMF using the pretrained models of GMF and MLP

### Training GMF

In [30]:
# Params
embedding_dim=200
lr=.01
regularization_constant=.01

# Test
gmf = GMF(num_users=X.shape[0], num_items=X.shape[1], embedding_dim=embedding_dim, regularization_constant=regularization_constant)
optimizer = torch.optim.Adam(gmf.parameters(), lr=lr)
gmf.to(device)
gmf.predict_single_interaction(0,0)

tensor([10.0000], device='cuda:7', grad_fn=<DiagonalBackward0>)

In [31]:
train_v2(train_X, valid_X, gmf, optimizer, n_epochs=25, batch_size=64, rmse=True)

100%|████████████████████████████████| 508/508 [00:01<00:00, 345.09it/s]


Epoch 1 MSE Loss: 15.794, valid MSE Loss: 9.327, valid RMSE Loss: 3.054, valid MAE Loss: 2.511


100%|████████████████████████████████| 508/508 [00:01<00:00, 304.11it/s]


Epoch 2 MSE Loss: 4.093, valid MSE Loss: 5.261, valid RMSE Loss: 2.294, valid MAE Loss: 1.89


100%|████████████████████████████████| 508/508 [00:01<00:00, 380.42it/s]


...

Epoch 23 MSE Loss: 1.599, valid MSE Loss: 3.772, valid RMSE Loss: 1.942, valid MAE Loss: 1.555


100%|████████████████████████████████| 508/508 [00:01<00:00, 300.59it/s]


Epoch 24 MSE Loss: 1.586, valid MSE Loss: 3.696, valid RMSE Loss: 1.923, valid MAE Loss: 1.53


100%|████████████████████████████████| 508/508 [00:01<00:00, 346.46it/s]


Epoch 25 MSE Loss: 1.568, valid MSE Loss: 3.744, valid RMSE Loss: 1.935, valid MAE Loss: 1.543


In [32]:
eval_model(test_X, gmf)

test MSE Loss: 3.688, test RMSE Loss: 1.921, test MAE Loss: 1.547


## Training MLP

In [33]:
embedding_dim=200
lr=1e-2
regularization_constant=.011

mlp = MLP(num_users=X.shape[0], num_items=X.shape[1], 
                    embedding_dim=embedding_dim, 
                    regularization_constant=regularization_constant,
                )
optimizer = torch.optim.Adam(mlp.parameters(), lr=lr)
mlp.to(device)
mlp.predict_single_interaction(0,0)

tensor(5.0489, device='cuda:7', grad_fn=<AddBackward0>)

In [ ]:
optimizer = torch.optim.Adam(mlp.parameters(), lr=lr)
train_v2(train_X, valid_X, mlp, optimizer, n_epochs=5, batch_size=64, rmse=True)

In [35]:
eval_model(test_X, mlp)

test MSE Loss: 2.906, test RMSE Loss: 1.705, test MAE Loss: 1.302


## Training NeuMF

### Hyper Params

In [37]:
# ## Decent set of hyper params
# MF_embedding_dim=200
# MLP_embedding_dim=200
# lr=1e-3
# regularization_constant=1e-2
# alpha = 0.5

# neumf = NeuMF(num_users=X.shape[0], num_items=X.shape[1], 
#                     MF_embedding_dim=MF_embedding_dim, 
#                     MLP_embedding_dim=MLP_embedding_dim,
#                     alpha=alpha,
#                     regularization_constant=regularization_constant,
#                 )

# optimizer = torch.optim.SGD(neumf.parameters(), lr=lr)
# neumf.to(device)
# neumf.predict_single_interaction(0,0)

tensor([8.0125, 3.4545], device='cuda:7', grad_fn=<AddBackward0>)

In [41]:
## Experimentation Params
MF_embedding_dim=200 # Must match gmf
MLP_embedding_dim=200 # Must match mlp
lr=1e-3
regularization_constant=1e-2
alpha = 0.3

neumf = NeuMF(num_users=X.shape[0], num_items=X.shape[1], 
                    MF_embedding_dim=MF_embedding_dim, 
                    MLP_embedding_dim=MLP_embedding_dim,
                    alpha=alpha,
                    regularization_constant=regularization_constant,
                )

optimizer = torch.optim.SGD(neumf.parameters(), lr=lr)
neumf.to(device)
neumf.predict_single_interaction(0,0)

tensor([6.9633, 6.4596], device='cuda:7', grad_fn=<AddBackward0>)

### Assign the pretrained weights

In [42]:
def init_pretrained(NeuMF_model, GMF_model, MLP_model):
    # Latent weights
    NeuMF_model.MF_user_factors.weight = GMF_model.user_factors.weight
    NeuMF_model.MF_item_factors.weight = GMF_model.item_factors.weight
    NeuMF_model.MLP_user_factors.weight = MLP_model.user_factors.weight
    NeuMF_model.MLP_item_factors.weight = MLP_model.item_factors.weight
    
    # MLP weights
    NeuMF_model.fc1.weight = MLP_model.fc1.weight
    NeuMF_model.fc2.weight = MLP_model.fc2.weight
    NeuMF_model.fc3.weight = MLP_model.fc3.weight
    
init_pretrained(neumf, gmf, mlp)

### Train!

In [43]:
train_v2(train_X, valid_X, neumf, optimizer, n_epochs=25, batch_size=64, rmse=True)

100%|████████████████████████████████| 508/508 [00:02<00:00, 188.93it/s]


Epoch 1 MSE Loss: 1.83, valid MSE Loss: 3.264, valid RMSE Loss: 1.807, valid MAE Loss: 1.435


100%|████████████████████████████████| 508/508 [00:02<00:00, 178.67it/s]


Epoch 2 MSE Loss: 1.659, valid MSE Loss: 3.128, valid RMSE Loss: 1.769, valid MAE Loss: 1.385


100%|████████████████████████████████| 508/508 [00:02<00:00, 182.96it/s]


...

Epoch 23 MSE Loss: 1.59, valid MSE Loss: 3.035, valid RMSE Loss: 1.742, valid MAE Loss: 1.343


100%|████████████████████████████████| 508/508 [00:02<00:00, 184.46it/s]


Epoch 24 MSE Loss: 1.591, valid MSE Loss: 3.036, valid RMSE Loss: 1.742, valid MAE Loss: 1.343


100%|████████████████████████████████| 508/508 [00:02<00:00, 183.55it/s]


Epoch 25 MSE Loss: 1.589, valid MSE Loss: 3.035, valid RMSE Loss: 1.742, valid MAE Loss: 1.343


## Evaluation

In [44]:
total_test_loss = eval_MSE_loss(test_X, neumf)
total_test_RMSE_loss = eval_RMSE_loss(test_X, neumf)
total_test_MAE_loss = eval_MAE_loss(test_X, neumf)
print(f"test MSE Loss: {total_test_loss}, test RMSE Loss: {total_test_RMSE_loss}, test MAE Loss: {total_test_MAE_loss}")

test MSE Loss: 2.931, test RMSE Loss: 1.712, test MAE Loss: 1.33
